In [36]:
from src.data_loader import PVDataLoader
from src import data_loader as dl
import torch
import numpy as np

In [37]:
# set seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# data parameters
DAY_INIT = 10
DAY_MIN = 8
DAY_MAX = 16
N_DAYS = 5
MINUTE_INTERVAL = 5
DAILY_DATA_POINTS = (DAY_MAX - DAY_MIN) * 60 / MINUTE_INTERVAL
N_HOURS_PRED = 2
N_SYSTEMS = 15
RADIUS = 0.35
COORDS = (55, -1.5)
IDX = 6

In [38]:
loader = PVDataLoader(n_days=N_DAYS,
                    day_init=DAY_INIT,
                    n_systems=N_SYSTEMS,
                    radius=RADIUS,
                    coords=COORDS,
                    minute_interval=MINUTE_INTERVAL,
                    day_min=DAY_MIN,
                    day_max=DAY_MAX,
                    folder_name='pv_data',
                    file_name_pv='pv_data_clean.csv',
                    file_name_location='location_data_clean.csv')

time, y = loader.get_time_series()
# scale time to be between 0 and 1

t_train, y_train, t_test, y_test = dl.train_test_split(time, y, n_hours=N_HOURS_PRED)

==> Loading data
==> Loaded data in: 0 m : 8 sec

==> Loading data
==> Loaded data in: 0 m : 0 sec



In [39]:
periodic_time = dl.periodic_mapping(time, DAY_MIN, DAY_MAX, minute_interval=MINUTE_INTERVAL)
periodic_train, _, periodic_test, _ = dl.train_test_split(periodic_time, y, n_hours=N_HOURS_PRED)

# Approximate Latent Force Model

In [40]:
import torch
import gpytorch
import numpy as np
from gpytorch.mlls.variational_elbo import VariationalELBO
from matplotlib import pyplot as plt
from os import path
from alfi.plot import Plotter1d
from lfm.dataset import PV_LFM_Dataset

In [41]:
num_outputs = 1
num_latents = 1
num_inducing = 100
dataset = PV_LFM_Dataset(num_outputs=num_outputs, 
                         m_observed=y_train[:, :num_outputs],
                         f_observed=periodic_train, 
                         train_t=t_train,
                         variance= 0.1 * torch.ones(num_outputs))

In [42]:
from alfi.models.variational_lfm import VariationalLFM, TrainMode
from alfi.models.ordinary_lfm import OrdinaryLFM
from alfi.configuration import VariationalConfiguration
from alfi.utilities.torch import is_cuda
import lfm

In [154]:
from abc import abstractmethod

import torch
import gpytorch
from torch.distributions import Distribution
from torchdiffeq import odeint
from gpytorch.lazy import DiagLazyTensor
from alfi.utilities.torch import is_cuda

class OrdinaryLFM(VariationalLFM):
    """
    Variational approximation for an LFM based on an ordinary differential equation (ODE).
    Inheriting classes must override the `odefunc` function which encodes the ODE.
    """

    def __init__(self,
                 num_outputs,
                 gp_model,
                 config: VariationalConfiguration,
                 initial_state=None,
                 **kwargs):
        super().__init__(num_outputs, gp_model, config, **kwargs)
        self.nfe = 0
        self.latent_gp = None
        
        if initial_state is None:
            self.initial_state = torch.zeros(torch.Size([self.num_outputs, 1]), dtype=self.dtype)
        else:
            self.initial_state = initial_state

    @property
    def initial_state(self):
        return self._initial_state

    @initial_state.setter
    def initial_state(self, value):
        value = value.cuda() if is_cuda() else value
        self._initial_state = value

    def forward(self, t, step_size=1e-1, return_samples=False, **kwargs):
        """
        t : torch.Tensor
            Shape (num_times)
        h : torch.Tensor the initial state of the ODE
            Shape (num_genes, 1)
        Returns
        -------
        Returns evolved h across times t.
        Shape (num_genes, num_points).
        """
        self.nfe = 0

        t_f = torch.arange(t.min(), t.max()+step_size/3, step_size/3)
        t_output = t
        h0 = self.initial_state
        h0 = h0.repeat(self.config.num_samples).reshape(1, self.config.num_samples, self.num_outputs)

        q_f = self.gp_model(t_f)
        
        self.latent_gp = q_f.rsample(torch.Size([self.config.num_samples])) #.permute(0, 2, 1)  # (S, I, T)
   
        # Integrate forward from the initial positions h0.
        self.t_index = 0
        self.last_t = t_f.min() - 1
        h_samples = odeint(self.odefunc, h0, t, method='rk4', options=dict(step_size=step_size)) # (T, S, num_outputs, 1)

        if return_samples:
            return h_samples

        dist = self.build_output_distribution(t_output, h_samples)
        self.latent_gp = None
        return dist

    def build_output_distribution(self, t, h_samples) -> Distribution:
        """
        Parameters:
            h_samples: shape (T, S, D)
        """
        h_mean = h_samples.mean(dim=1).squeeze(-1).transpose(0, 1)  # shape was (#outputs, #T, 1)
        h_var = h_samples.var(dim=1).squeeze(-1).transpose(0, 1) + 1e-7

        # TODO: make distribution something less constraining
        # TODO possibly make a beta distribution by sampling from posterior
        if self.config.latent_data_present:
            # todo: make this
            f = self.gp_model(t).rsample(torch.Size([self.config.num_samples])).permute(0, 2, 1)
            f_mean = f.mean(dim=0)
            f_var = f.var(dim=0) + 1e-7
            h_mean = torch.cat([h_mean, f_mean], dim=0)
            h_var = torch.cat([h_var, f_var], dim=0)

        h_covar = DiagLazyTensor(h_var)  # (num_tasks, t, t)
        batch_mvn = gpytorch.distributions.MultivariateNormal(h_mean, h_covar)
        return gpytorch.distributions.MultitaskMultivariateNormal.from_batch_mvn(batch_mvn, task_dim=0)

    @abstractmethod
    def odefunc(self, t, h, **kwargs):
        """
        Parameters:
            h: shape (num_samples, num_outputs, 1)
        """
        pass

    def sample_latents(self, t, num_samples=1):
        q_f = self.gp_model(t)
        return self.nonlinearity(q_f.sample(torch.Size([num_samples])))


In [167]:
class PhotovoltaicLFM(OrdinaryLFM):
    def __init__(self, 
                num_outputs : int, 
                gp_model : gpytorch.models.ApproximateGP, # multi-task GP 
                config: gpytorch.variational.VariationalStrategy,
                dataset : lfm.dataset.LFMDataset,
                nonlinear : bool = False,
                **kwargs):
        
        super().__init__(num_outputs=num_outputs, 
                         gp_model=gp_model, 
                         config=config, 
                         initial_state=dataset.data[0][1][0], 
                         **kwargs)
        
        self.raw_initial_pv = dataset.data[0][1][0]
        self.nonlinear = nonlinear
        self.true_f = dataset.f_observed
        self.raw_decay = torch.nn.Parameter(torch.randn(num_outputs, 1))
        self.raw_growth = torch.nn.Parameter(torch.randn(num_outputs, 1))
        
    @property
    def initial_pv(self):
        return self.raw_initial_pv
    
    @initial_pv.setter
    def initial_pv(self, value):
        value = value.cuda() if is_cuda() else value
        self.raw_initial_pv = value
    
    def initial_state(self):
        return self.initial_pv
    
    @property
    def decay(self):
        return self.raw_decay
    
    @decay.setter
    def decay(self, value):
        value = value.cuda() if is_cuda() else value
        self.raw_decay = value
    
    @property
    def growth(self):
        return self.raw_growth
    
    @growth.setter
    def growth(self, value):
        value = value.cuda() if is_cuda() else value
        self.raw_growth = value
    
    def G(self, f):
        if self.nonlinear:
            return torch.sigmoid(f).repeat(1, self.num_outputs, 1)
        else:
            return f

    def odefunc(self, t, h):
        # TODO ensure that the time passed to 
        f_latents = self.G(self.latent_gp)
        print(f'h shape: {h.shape}')
        print(f'f_latents shape: {f_latents.shape}')
        print(f'growth shape: {self.growth.shape}')
        print(f'decay shape: {self.decay.shape}')
        print(f't shape: {t.shape}')
        dh = self.growth * h * f_latents - self.decay * h
        return dh 


In [168]:
from src.models import ApproximateGPBaseModel, MultiTaskBetaGP
from lfm.trainers import VariationalTrainer
from src.beta_likelihood import BetaLikelihood_MeanParametrization


# Kernels

In [169]:
from gpytorch.kernels import (MaternKernel, 
                              PeriodicKernel,
                              RBFKernel,
                              ScaleKernel, 
                              AdditiveKernel, 
                              ProductKernel)
# TODO add a prior to the period in periodic

matern_base = MaternKernel(nu=3/2, 
                      lengthscale_prior=gpytorch.priors.GammaPrior(2, 8),
                      lengthscale_constraint=gpytorch.constraints.Positive()
                      )
periodic = PeriodicKernel(period_length_prior=gpytorch.priors.GammaPrior(3, 2),
                            period_length_constraint=gpytorch.constraints.Positive()
                        )
scaled_periodic = ScaleKernel(periodic,
                                outputscale_prior=gpytorch.priors.GammaPrior(5, 1),
                                outputscale_constraint=gpytorch.constraints.Positive()
                            )
scaled_matern = ScaleKernel(matern_base, 
                            outputscale_prior=gpytorch.priors.GammaPrior(5, 2),
                            outputscale_constraint=gpytorch.constraints.Interval(0.1, 1)
                            )
product_kernel_matern_periodic = ScaleKernel(periodic * matern_base,
                             outputscale_prior = gpytorch.priors.GammaPrior(5, 2),
                             outputscale_constraint=gpytorch.constraints.Positive()
                            )
rbf_kernel = ScaleKernel(RBFKernel())
product_kernel_rbf_matern = ScaleKernel(rbf_kernel * scaled_matern)


quasi_periodic_rbf = AdditiveKernel(product_kernel_rbf_matern, scaled_matern)
quasi_periodic_matern = AdditiveKernel(product_kernel_matern_periodic, scaled_matern)

#### Ensure that the trainer will receive the parameters from both the interaction terms and the latent GP paramters

In [170]:
jitter = 1e-4
gp_config = {
            'type': 'stochastic',
            'name': 'mean_field',
            'num_inducing_points': t_train.size(0),
            'mean_init_std': 1,
            }
gp_inputs = {
            'X': dataset.t_observed, 
            'y': dataset.data[0][1], 
            'mean_module': gpytorch.means.ZeroMean(),
            'covar_module': quasi_periodic_matern,
            'likelihood': BetaLikelihood_MeanParametrization(scale=10,
                                                                correcting_scale=1,
                                                                lower_bound=0.10,
                                                                upper_bound=0.80),
            # 'num_latents' : num_latents,
            # 'variational_strategy': 'mt_indep',
            'config': gp_config,
            'jitter': jitter
}
gp_model = ApproximateGPBaseModel(**gp_inputs)

In [171]:
from alfi.configuration import VariationalConfiguration
config = VariationalConfiguration(num_samples=10)

lfm_model = PhotovoltaicLFM(num_outputs=1,
                            gp_model=gp_model,
                            config=config,
                            dataset=dataset,
                            nonlinear=True)

In [172]:
out = lfm_model(t_train, step_size=1e-2)

h shape: torch.Size([1, 10, 1])
f_latents shape: torch.Size([1, 10, 456])
growth shape: torch.Size([1, 1])
decay shape: torch.Size([1, 1])
t shape: torch.Size([])
h shape: torch.Size([1, 10, 456])
f_latents shape: torch.Size([1, 10, 456])
growth shape: torch.Size([1, 1])
decay shape: torch.Size([1, 1])
t shape: torch.Size([])
h shape: torch.Size([1, 10, 456])
f_latents shape: torch.Size([1, 10, 456])
growth shape: torch.Size([1, 1])
decay shape: torch.Size([1, 1])
t shape: torch.Size([])
h shape: torch.Size([1, 10, 456])
f_latents shape: torch.Size([1, 10, 456])
growth shape: torch.Size([1, 1])
decay shape: torch.Size([1, 1])
t shape: torch.Size([])


RuntimeError: The expanded size of the tensor (1) must match the existing size (456) at non-singleton dimension 2.  Target sizes: [1, 10, 1].  Tensor sizes: [10, 456]